# Profiling
## Toy problem: read in 5000 words and find duplicates (case-insensitive)
### Which parts of code use most execution time?
Video: https://www.youtube.com/watch?v=8qEnExGLZfY

In [27]:
# first, generate txt w/ 5000 lines/strings
import random, string


with open('words.txt', 'w') as f:
    for _ in range(5000):
        word = ''.join(random.choice(string.ascii_letters) for _ in range(3))
        f.write(word + '\n')

## Initial, inefficient solution

In [33]:
# read in words
def read_words(file):
    with open(file) as f:
        return f.read().splitlines()

In [34]:
# check if duplicate
def is_duplicate(needle, haystack):
    for word in haystack:
        if needle.lower() == word.lower():
            return True
    return False

In [39]:
# find duplicates (case-insensitive)
def find_duplicates(file):
    haystack = read_words(file)
    duplicates = []
    
    while haystack:
        needle = haystack.pop()
        if is_duplicate(needle, haystack):
            duplicates.append(needle)
            
    return duplicates

In [36]:
# dups = find_duplicates('words.txt')
# len(dups)

656

### Profiling
docs: https://docs.python.org/3/library/profile.html#profile.Profile

In [37]:
# Profiling
import cProfile

cProfile.run('find_duplicates("words.txt")')

# 4.053 seconds, 4.050 in is_duplicate()!, 1.581 in lower() (also, lower() was called 23M times)

         22807860 function calls in 4.053 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-33-95314b026153>:2(read_words)
     5000    2.469    0.000    4.050    0.001 <ipython-input-34-4969f528cf04>:2(is_duplicate)
        1    0.002    0.002    4.053    4.053 <ipython-input-35-fbf4261734c0>:2(find_duplicates)
        1    0.000    0.000    4.053    4.053 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method _locale.nl_langinfo}
        1    0.000    0.000    4.0

## Optimizing

### 1) call `lower()` when read in words, reduce calls from ~25M to just 1!

In [41]:
def read_words(file):
    with open(file) as f:
        return f.read().lower().splitlines()
    
# check if duplicate
def is_duplicate(needle, haystack):
    for word in haystack:
        if needle == word:
            return True
    return False

In [42]:
cProfile.run('find_duplicates("words.txt")')

# >10X speedup! 

         10671 function calls in 0.298 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.298    0.298 <ipython-input-39-fbf4261734c0>:2(find_duplicates)
        1    0.000    0.000    0.001    0.001 <ipython-input-41-1bafda4d4f99>:1(read_words)
     5000    0.295    0.000    0.295    0.000 <ipython-input-41-1bafda4d4f99>:6(is_duplicate)
        1    0.000    0.000    0.298    0.298 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method _locale.nl_langinfo}
        1    0.000    0.000    0.298 

### 2) calling `is_duplicate()` is time consuming (0.295/0.298), its simple now so just remove it

In [43]:
# find duplicates (case-insensitive)
def find_duplicates(file):
    haystack = read_words(file)
    duplicates = []
    
    while haystack:
        needle = haystack.pop()
        if needle in haystack:
            duplicates.append(needle)
            
    return duplicates

In [44]:
cProfile.run('find_duplicates("words.txt")')

# ~2X speedup (.298 => .169)

         5671 function calls in 0.169 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-41-1bafda4d4f99>:1(read_words)
        1    0.168    0.168    0.169    0.169 <ipython-input-43-6f48b450ebec>:2(find_duplicates)
        1    0.000    0.000    0.169    0.169 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method _locale.nl_langinfo}
        1    0.000    0.000    0.169    0.169 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-i

### 3) Algorithmic adjustment (using hash table instead of iterating through list for each word)

`O(N^2) => O(N)`

In [51]:
# find duplicates (case-insensitive)
def find_duplicates(file):
    haystack = read_words(file)
    duplicates = []
    
    # store words in hash table (O(N))
    hash_table = {}
    for word in haystack:
        # lookup is O(1)
        result = hash_table.get(word)
        if result is None:
            hash_table[word] = 1
        else:
            duplicates.append(word)
            
    return duplicates

In [54]:
cProfile.run('find_duplicates("words.txt")')

# 56X speedup! (.169 => .003)

         5671 function calls in 0.003 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-41-1bafda4d4f99>:1(read_words)
        1    0.001    0.001    0.003    0.003 <ipython-input-51-ceacab52f32b>:2(find_duplicates)
        1    0.000    0.000    0.003    0.003 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method _locale.nl_langinfo}
        1    0.000    0.000    0.003    0.003 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-i

### Overall Performance Boost:

Theoretically: `O(N^2) => O(N)`

Time: `4.053 => .003` which is `1350X` speedup